In [1]:
import argparse
import os

if hasattr(__builtins__,'__IPYTHON__'):
    
    sysName='DY'
    sysfiles='/eos/cms/store/group/phys_egamma/akapoor/ChargeMisID/newsamples2/06544C90-EFDF-E811-80E6-842B2B6F5D5C.root'
    saveRootFile=True
    outfolder='Output'
    outsuffix='1'
    debugprint=True
    analysis='TTH'
    multithreaded=True
    xsec=1.00
    istype='mc'
    #branchlist="(ChargeFlip_Category|MyElectron_pt)"
else:
    parser = argparse.ArgumentParser(description='Options for the analyzer')
    parser.add_argument('--name', metavar='n', action='store', type=str,help='short name of the physics process/Data sample', required=True)
    parser.add_argument('--file', metavar='f', action='store', type=str,help='location of files',required=True)
    parser.add_argument('--saveroot',action='store_true', help='save root file',default=False)
    parser.add_argument('--debugprint',action='store_true', help='print analysis settings for debug',default=False)
    parser.add_argument('--outfolder', metavar='of', action='store', type=str,help='location of output',default='Output',required=True)
    parser.add_argument('--outsuffix', metavar='os', action='store', type=str,help='suffix to recognize job',default='Output')
    parser.add_argument('--analysis', metavar='ana', action='store', type=str,help='Folder name that has analysis configs',default='Output',required=True)
    parser.add_argument('--multithreaded', action='store_true', help='save root file',default=False,required=True )
    parser.add_argument('--xsec', metavar='xsec', action='store', type=float, help='xsec',default=1,required=True )
    parser.add_argument('--istype', metavar='istype', action='store', type=str, help='data or mc',default='mc',required=True )
    #parser.add_argument('--branchlist', metavar='bl', action='store', type=str,help='branches to store in skim',default="(MyElectron_eta|MyElectron_pt)")
    args = parser.parse_args()
    
    sysName=str(args.name)
    sysfiles=str(args.file)
    saveRootFile=args.saveroot
    outfolder=str(args.outfolder)
    outsuffix=str(args.outsuffix)
    analysis=str(args.analysis)
    debugprint=args.debugprint
    multithreaded=args.multithreaded
    xsec=args.xsec
    istype=args.istype
    #branchlist=str(args.branchlist)

os.system("mkdir "+outfolder)

mkdir: cannot create directory ‘Output’: File exists


256

In [2]:
import ROOT
if multithreaded:
    ROOT.ROOT.EnableImplicitMT()

ROOT.gInterpreter.AddIncludePath("./include/");
ROOT.gInterpreter.ProcessLine('#include "Helper.h"')
from include.PyHelper import *

Welcome to JupyROOT 6.24/06


Warning in <RTaskArenaWrapper>: CPU Bandwith Control Active. Proceeding with 4 threads accordingly


In [3]:
import yaml
ObjectCutsFile=analysis+"/ObjectCuts.yaml"
DefinitionsFile=analysis+"/Definitions.yaml"
SelectionsFile=analysis+"/Selections.yaml"
BranchesToSaveFile=analysis+"/BranchesToSave.yaml"

ObjectCuts = yaml.safe_load(open(ObjectCutsFile, 'r'))
Definitions = yaml.safe_load(open(DefinitionsFile, 'r'))
Selections = yaml.safe_load(open(SelectionsFile, 'r'))
BranchesToSave = yaml.safe_load(open(BranchesToSaveFile, 'r'))

In [4]:
Ana= MyAnalyzer()
for key in ObjectCuts.keys():
    exec(f'{key}mask=ObjectMask()')
    for cut in ObjectCuts[key]:
        exec(f'{key}mask.addcut("{cut}")')
    exec(f'mask={key}mask.mask')
    exec(f'Ana.definition("{key}",{key}mask.mask)')
for key in Definitions.keys():
    Ana.definition(key,Definitions[key])
for key in Selections.keys():
    Ana.sel(key,Selections[key])
    
    
processname=sysName
processdict={'Files':sysfiles,'Type':istype,'Tree':'Events','Xsec':xsec}
Ana.process(processname,processdict)

In [5]:
Total=Ana.prepare()

Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


In [6]:
DF=Ana.PD[sysName]['RDF']

In [7]:
import json
if debugprint:
    
    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print('Dumping the analysis graph')
    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print(f'Object Mask Cuts from {ObjectCutsFile}')
    print(json.dumps(ObjectCuts, indent=3))
    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print(f'All the definitions from {DefinitionsFile}')
    print(json.dumps(Definitions, indent=3))
    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print(f'All the Event Selections from {SelectionsFile}')
    print(json.dumps(Selections, indent=3))
    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print(f'Sample information provided as options')
    print(json.dumps(processdict, indent=3))
    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print(f'Branches to save taken from {BranchesToSaveFile}')
    print(json.dumps(BranchesToSave, indent=3))
    print('---------------------------------------------------')
    print('---------------------------------------------------')
    if not hasattr(__builtins__,'__IPYTHON__'):
        print('Skimmer options used')
        print(args)
    print('---------------------------------------------------')
    print('---------------------------------------------------')

---------------------------------------------------
---------------------------------------------------
Dumping the analysis graph
---------------------------------------------------
---------------------------------------------------
Object Mask Cuts from TTH/ObjectCuts.yaml
{
   "GoodElectrons": [
      "Electron_pt > 10",
      "Electron_sieie<(0.011+0.019*(abs(Electron_eta+Electron_deltaEtaSC)>1.479))",
      "abs(Electron_dxy)< 0.05",
      "abs(Electron_dz)< 0.1",
      "Electron_mvaTTH > 0.8",
      "Electron_hoe<0.1",
      "Electron_eInvMinusPInv>-0.04",
      "Electron_sip3d<8",
      "Electron_convVeto",
      "Electron_miniPFRelIso_all<0.40",
      "Electron_lostHits==0"
   ]
}
---------------------------------------------------
---------------------------------------------------
All the definitions from TTH/Definitions.yaml
{
   "Nel": "Sum(GoodElectrons)",
   "elpt": "Take(Electron_pt[GoodElectrons], Reverse(Argsort(Electron_pt[GoodElectrons])))",
   "eleta": "Take(Electr

In [14]:
#DF.Display({"event","elpt","eldeepjet"}).Print()
cutflow=DF.Report()
Final=DF.Count()

In [16]:
print('Saving the Action Graph')
ROOT.RDF.SaveGraph(cutflow, f'{outfolder}/ActionGraph.dot')
from graphviz import render
render('dot', 'png', f'{outfolder}/ActionGraph.dot')

Saving the Action Graph


'Output/ActionGraph.dot.png'

In [ ]:
import sys
if saveRootFile:
    BRS=BranchesToSave.replace(" ", "")
    outfilenow=outfolder+"/output_"+outsuffix+".root"
    snapshot=DF.Snapshot("Events",outfilenow,f"{BRS}")
    #fdf=DF.Snapshot("Events",outfilenow,f"{BRS}")
    #if debugprint:
    cutflow.Print()
    #sys.stdout = open(outfolder+"/CutFlow.csv", "a")
    #print(f'{sysfiles}' + "\n")
    
    #sys.stdout.close()
    TotalEvents=Total.GetValue()
    FinalEvents=Final.GetValue()
    with open(outfolder+"/Info.csv", "a") as text_file:
            #print(driver.current_url)
            text_file.write(f'{sysfiles}, {istype}, {xsec}, {FinalEvents}, {TotalEvents}, {round((FinalEvents*100)/TotalEvents,2)}%, {outfilenow}' + "\n")